## Method2
#### Use Gram weight as the recipe-ingredient matrix values

### Process data

In [9]:
import pandas as pd
import numpy as np

In [25]:
class recipe_ingredient_data(object):
    def __init__(self, ingredient_path = "dataset/ingredient.csv", reciepe_path = "dataset/recipe.csv"):
        self.ingredient_path = ingredient_path
        self.reciepe_path = reciepe_path
        self.read_data()
        
    def read_data(self):
        self.ingredient_df = pd.read_csv(self.ingredient_path)
        self.recipe_df = pd.read_csv(self.reciepe_path)
        print("Finsh reading raw ingredient and recipe data")
    
    def generate_recipe_ing_map(self,col_Name):
        # generate map {'recipe_id': {[ingredient1,ingredient2... ]}}
        recipe_ing_map = {}
        for ind, row in self.ingredient_df.iterrows(): 
            if row['recipe_id'] in recipe_ing_map.keys():
                recipe_ing_map.get(row['recipe_id']).add(row[col_Name])
            else:
                recipe_ing_map[row['recipe_id']] = set([row[col_Name]])
        print("Finish generating map for column: "+col_Name)
        return recipe_ing_map
    
    def generate_idx_maps(self):
        ## generate a unique index ing_map and recipe map
        ing_lst = np.sort(self.ingredient_df["nutrition_key"].unique()).tolist()
        recipe_lst =  list(self.ingredient_df["recipe_id"].unique())
        self.rmap = {x: i for i, x in enumerate(recipe_lst)}
        self.imap = {x: i for i, x in enumerate(ing_lst)}
        self.rmap_reversed =  {v: k for k, v in self.rmap.items()}

        print("number of total ingrediants in imap: "+ str(len(self.imap)))
        print("number of unqiue recipes in imap: "+ str(len(self.rmap)))
    
    def generate_RImatrix(self):
        one_hot = pd.DataFrame(columns=["recipe_id", "ingredients"])
        one_hot["recipe_id"] = self.ingredient_df["recipe_id"].map(self.rmap)
        one_hot["ingredients"] = self.ingredient_df["nutrition_key"].drop(columns=['zzseasoning'])
        one_hot["weight_g"] =  self.ingredient_df["weight_g"].values.astype(np.float32)
        ingMatrix = pd.pivot_table(one_hot, index=['recipe_id'], columns=['ingredients'], values=['weight_g'],fill_value=0)
        self.ingMatrix = ingMatrix.drop(columns=[('weight_g','zzseasoning'),('weight_g','salt'),('weight_g','oil')]) # zzseasoning, salt, oil
        print("Finish generating recipe-ingredient matrix")
        
    def prepare_data(self): # main
        # prepare all the data and maps
        recipe_map = self.generate_recipe_ing_map(col_Name='nutrition_key')
        recipe_map_raw =  self.generate_recipe_ing_map(col_Name='ingredient')
        # Read as pandas df
        self.recipe_ing_df = pd.DataFrame(data = recipe_map.items(),columns=['recipe_id','ingredients'])
        self.recipe_ing_raw_df = pd.DataFrame(data = recipe_map_raw.items(),columns=['recipe_id','ingredients_raw'])
        # convert from set to list for ingredients
        self.recipe_ing_df.ingredients = self.recipe_ing_df.ingredients.apply(list)
        self.recipe_ing_raw_df.ingredients_raw = self.recipe_ing_raw_df.ingredients_raw.apply(list)
        # generate index mappings
        self.generate_idx_maps()
        # generate recipe-ingredient matrix, weight as the values
        self.generate_RImatrix()
    

In [45]:
data = recipe_ingredient_data()
data.prepare_data()
data.recipe_ing_df

Finsh reading raw ingredient and recipe data
Finish generating map for column: nutrition_key
Finish generating map for column: ingredient
number of total ingrediants in imap: 430
number of unqiue recipes in imap: 2256
Finish generating recipe-ingredient matrix


,recipe_id,ingredients
0,35169,"[cream, chicken, celery, carrot, blue cheese, ..."
1,35171,"[chicken, celery, carrot, cheddar cheese, butt..."
2,47893,"[berries, fennel, kale, avocado, lemon juice, ..."
3,36259,"[broth, mixed vegetables, salt, butter, potato..."
4,47233,"[wine, zzseasoning, oregano, carrot, parmesan ..."
...,...,...
2251,50331,"[wine, broth, zzseasoning, vinegar, corn, cele..."
2252,54683,"[zzseasoning, chicken, celery, carrot, water, ..."
2253,055eb1,"[wine, corn, balsamic vinegar, broth, pot pies..."
2254,f69f3e,"[wine, corn, parsley, oil, cherry tomatoes, ce..."


In [46]:
data.ingMatrix

weight_g                                            \
ingredients agave nectar alfalfa almond butter almond milk almonds   
recipe_id                                                            
0                    0.0       0           0.0         0.0     0.0   
1                    0.0       0           0.0         0.0     0.0   
2                    0.0       0           0.0         0.0    20.0   
3                    0.0       0           0.0         0.0     0.0   
4                    0.0       0           0.0         0.0     0.0   
...                  ...     ...           ...         ...     ...   
2251                 0.0       0           0.0         0.0     0.0   
2252                 0.0       0           0.0         0.0     0.0   
2253                 0.0       0           0.0         0.0     0.0   
2254                 0.0       0           0.0         0.0     0.0   
2255                 0.0       0           0.0         0.0     0.0   

                                                                     ...  \
ingredients american cheese anchovies apple apple cider apple juice  ...   
recipe_id                                                            ...   
0                       0.0         0   0.0         0.0         0.0  ...   
1                       0.0         0   0.0         0.0         0.0  ...   
2                       0.0         0   0.0         0.0         0.0  ...   
3                       0.0         0   0.0         0.0         0.0  ...   
4                       0.0         0   0.0         0.0         0.0  ...   
...                     ...       ...   ...         ...         ...  ...   
2251                    0.0         0   0.0         0.0         0.0  ...   
2252                    0.0         0   0.0         0.0         0.0  ...   
2253                    0.0         0   0.0         0.0         0.0  ...   
2254                    0.0         0   0.0         0.0         0.0  ...   
2255                    0.0         0   0.0         0.0         0.0  ...   

                                                                           \
ingredients water chestnuts watermelon wheat beer wheat berry wheat bread   
recipe_id                                                                   
0                       0.0        0.0        0.0         0.0           0   
1                       0.0        0.0        0.0         0.0           0   
2                       0.0        0.0        0.0         0.0           0   
3                       0.0        0.0        0.0         0.0           0   
4                       0.0        0.0        0.0         0.0           0   
...                     ...        ...        ...         ...         ...   
2251                    0.0        0.0        0.0         0.0           0   
2252                    0.0        0.0        0.0         0.0           0   
2253                    0.0        0.0        0.0         0.0           0   
2254                    0.0        0.0        0.0         0.0           0   
2255                    0.0        0.0        0.0         0.0           0   

                                                            
ingredients whole wheat bread   wine wraps yogurt zucchini  
recipe_id                                                   
0                         0.0    0.0     0    0.0      0.0  
1                         0.0    0.0     0    0.0      0.0  
2                         0.0    0.0     0    0.0      0.0  
3                         0.0    0.0     0    0.0      0.0  
4                         0.0   20.0     0    0.0      0.0  
...                       ...    ...   ...    ...      ...  
2251                      0.0   25.0     0    0.0      0.0  
2252                      0.0    0.0     0    0.0      0.0  
2253                      0.0  750.0     0    0.0      0.0  
2254                      0.0   20.0     0    0.0      0.0  
2255                      0.0    0.0     0    0.0      0.0  

[2256 rows x 427 columns]

### Recommendation

In [28]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances

In [29]:
def calculate_scores(rec_idx, ingMatrix, sim = "cosine_similarity"):
    current_recipe = ingMatrix.loc[rec_idx].values.reshape(1, -1)
    if sim == "cosine_similarity":
        print("similarity function used: cosine_similarity")
        scores = [cosine_similarity(current_recipe, ingMatrix.loc[i].values.reshape(1, -1))[0][0] for i in ingMatrix.index] 
    else:
        print("similarity function used: euclidean_distances")
        scores = [1/(1 + euclidean_distances(current_recipe, ingMatrix.loc[i].values.reshape(1, -1)))[0][0] for i in ingMatrix.index] 

    return scores

In [38]:
def get_topN_recommendations(data, N, scores, exclude=False):
    # Get the highest N scores, if RR; then exclude itself
    if exclude:
        top = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)[1:N+1]
    else:
        top = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)[:N]
    
    # create dataframe to load in recommendations
    recommendation = pd.DataFrame(columns=["score","recipe_id","recipe", "ingredients", "sum_cal","sum_fat","sum_carb","sum_protein", "url"])
    count = 0
    print(top)
    for i in top: 
        recipe_id = data.rmap_reversed.get(i)
        recommendation.at[count, "recipe_id"] = recipe_id
        recommendation.at[count, "score"] = round(scores[i],5)
        recommendation.at[count, "ingredients"] = data.recipe_ing_raw_df['ingredients_raw'][i]       
        
        recipe_row_ind = data.recipe_df.loc[data.recipe_df['recipe_id'] == recipe_id].index[0]
        recommendation.at[count, "recipe"] = data.recipe_df["title"][recipe_row_ind]
        recommendation.at[count, "url"] = data.recipe_df["source_url"][recipe_row_ind]
        recommendation.at[count, "sum_cal"] = data.recipe_df["sum_cal"][recipe_row_ind]
        recommendation.at[count, "sum_carb"] = data.recipe_df["sum_carb"][recipe_row_ind]
        recommendation.at[count, "sum_protein"] = data.recipe_df["sum_protein"][recipe_row_ind]
        
        count += 1
    return recommendation

In [39]:
from datetime import datetime

def get_recommendation_RR(data,recipe_id, N=5,sim = "cosine_similarity",generate_json=False):
    print("Generating similar recipes for recipe:" + list(data.recipe_df.loc[data.recipe_df['recipe_id'] == recipe_id]['title'])[0])
    input_ings = list(data.recipe_ing_df.loc[data.rmap.get(recipe_id)].ingredients)
    print(input_ings)

    # get cosine similarity between current recipe embedding and other recipes
    scores = calculate_scores(ingMatrix=data.ingMatrix, sim = sim, rec_idx = data.rmap.get(recipe_id))
    
    # Filter top N recommendations
    recommendations = get_topN_recommendations(data, N, scores, exclude=True)
    
    if generate_json:
        ts = datetime.now().strftime("%Y_%m_%d_%H_%M")
        json_file_nm = "RR_results_"+ts+".json"
        recommendations.to_json(json_file_nm,orient='records',indent=2)
        print("Generated "+ json_file_nm)
    return recommendations

In [42]:
get_recommendation_RR(data,"2581e6") # cosine_similarity

Generating similar recipes for recipe:Cowboy Quiche
['pie crust', 'cheddar cheese', 'bacon', 'pepper', 'butter', 'cream', 'onions', 'eggs']
similarity function used: cosine_similarity
[460, 1567, 131, 1122, 76]


,score,recipe_id,recipe,ingredients,sum_cal,sum_fat,sum_carb,sum_protein,url
0,0.9115,35533,"Roast Turkey Sandwich with Artichoke Tapenade,...","[roast turkey, water, olive oil, large onion, ...",275.004,NaN,33.3776,10.4456,http://www.closetcooking.com/2012/03/roast-tur...
1,0.9065,38403,Lamb Kebabs With Lima Bean Salad,"[medium red onion, pitted kalamata olives, fro...",631.367611,NaN,86.826781,26.709664,http://www.realsimple.com/food-recipes/browse-...
2,0.90614,36561,Mango Salsa,"[medium red onion, fresh cilantro leaves, fres...",107.7172,NaN,24.7478,4.3981,http://www.simplyrecipes.com/recipes/mango_salsa/
3,0.89978,5993,Caribbean Fruity Salsa,"[medium red onion, diced fresh pineapple, dice...",166.802,NaN,29.5005,4.3563,http://allrecipes.com/Recipe/Caribbean-Fruity-...
4,0.8843,35540,Roasted Cauliflower and Aged White Cheddar Soup,"[medium onion, salt and pepper, oil, aged whit...",496.7676,NaN,45.38,16.776,http://www.closetcooking.com/2012/02/roasted-c...


In [40]:
get_recommendation_RR(data,"2581e6", sim ='euc') #euclidean_distances

Generating similar recipes for recipe:Cowboy Quiche
['pie crust', 'cheddar cheese', 'bacon', 'pepper', 'butter', 'cream', 'onions', 'eggs']
similarity function used: euclidean_distances
[460, 131, 1122, 76, 348]


,score,recipe_id,recipe,ingredients,sum_cal,sum_fat,sum_carb,sum_protein,url
0,0.01101,35533,"Roast Turkey Sandwich with Artichoke Tapenade,...","[roast turkey, water, olive oil, large onion, ...",275.004,NaN,33.3776,10.4456,http://www.closetcooking.com/2012/03/roast-tur...
1,0.01049,36561,Mango Salsa,"[medium red onion, fresh cilantro leaves, fres...",107.7172,NaN,24.7478,4.3981,http://www.simplyrecipes.com/recipes/mango_salsa/
2,0.01024,5993,Caribbean Fruity Salsa,"[medium red onion, diced fresh pineapple, dice...",166.802,NaN,29.5005,4.3563,http://allrecipes.com/Recipe/Caribbean-Fruity-...
3,0.00972,35540,Roasted Cauliflower and Aged White Cheddar Soup,"[medium onion, salt and pepper, oil, aged whit...",496.7676,NaN,45.38,16.776,http://www.closetcooking.com/2012/02/roasted-c...
4,0.00905,983c2a,Loaded Veggie and Prosciutto Pizza,"[artichoke hearts, salt and pepper, thinly sli...",703.258457,NaN,103.39437,20.749837,http://whatsgabycooking.com/loaded-veggie-and-...


### Pre-comput Top N for all recipes

In [51]:
import json
def get_recommendation_RR_ALL(data):
    start = datetime.now()
    recommendations = pd.DataFrame(columns=["score","recipe_id","recipe", "ingredients", "sum_cal","sum_fat","sum_carb","sum_protein", "url","current_recipe"])
    for k,v in data.rmap_reversed.items():
        recommendation = get_recommendation_RR(data, v)
        recommendation['current_recipe'] = v
        recommendations = pd.concat([recommendations,recommendation], ignore_index=True)
    stop = datetime.now()
    print("Total time used: "+ str(stop-start))
    
    res = recommendations.to_json(orient='records')
    parsed = json.loads(res)
    
    # convert to reciepe_id based 
    all_recipes = {}
    for i in parsed:
        if i['current_recipe'] in all_recipes.keys():
            all_recipes.get(i['current_recipe']).append(i)
        else:
            all_recipes[i['current_recipe']] = [i]
    
    ts = datetime.now().strftime("%Y_%m_%d_%H_%M")
    json_file_nm = "RR_results_"+ts+".json"
    with open(json_file_nm, "w") as outfile:
        json.dump(all_recipes, outfile, indent=4) 
    print("All recipes are being processed!")

In [252]:
start = datetime.now()
print("starting time:"+ str(start ))
recommendations = pd.DataFrame(columns=["score","recipe_id","recipe", "ingredients", "sum_cal","sum_fat","sum_carb","sum_protein", "url","current_recipe"])
for k,v in rmap_reversed.items():
    recommendation = get_recommendation_RR(data,v)
    recommendation['current_recipe'] = v
    recommendations = pd.concat([recommendations,recommendation], ignore_index=True)
stop = datetime.now()
print("stopping time:"+ sr(stop) )

starting time:2022-04-24 17:31:19.716513
['flour', 'blue cheese', 'butter', 'chicken', 'celery', 'cream', 'carrot', 'potatoes', 'barbecue sauce', 'garlic', 'onions', 'pepper']
[447, 2244, 961, 1828, 190]
['blue cheese', 'butter', 'chicken', 'celery', 'carrot', 'cheddar cheese', 'bread', 'mayonnaise', 'barbecue sauce', 'onions']
[2194, 2179, 213, 0, 571]
['salt', 'olive oil', 'berries', 'honey', 'lemon juice', 'avocado', 'carrot', 'fennel', 'almonds', 'kale', 'garlic', 'pepper']
[1235, 1686, 143, 1752, 840]
['salt', 'butter', 'beef', 'potatoes', 'barbecue sauce', 'mixed vegetables', 'onions', 'broth']
[238, 16, 2253, 1513, 1745]
['olive oil', 'wine', 'oregano', 'basil', 'carrot', 'ground beef', 'zzseasoning', 'parmesan cheese', 'milk', 'tomatoes', 'garlic', 'onions', 'pepper']
[1492, 1747, 1500, 229, 177]
['salt', 'olive oil', 'thyme', 'carrot', 'beef', 'rosemary', 'roast beef', 'wine', 'onions', 'pepper']
[460, 1567, 131, 1122, 1341]
['flour', 'salt', 'butter', 'sour cream', 'chicken',

[748, 982, 1413, 352, 1632]
['flour', 'butter', 'asparagus', 'bacon', 'leeks', 'zzseasoning', 'eggs', 'milk', 'pepper']
[1637, 74, 65, 49, 72]
['olive oil', 'asparagus', 'parmesan cheese', 'lemon', 'pepper']
[82, 654, 1768, 468, 58]
['olive oil', 'asparagus', 'cherry tomatoes', 'balsamic vinegar', 'salmon', 'basil', 'potatoes']
[425, 1841, 1629, 1628, 2249]
['olive oil', 'asparagus', 'avocado', 'seeds', 'tofu', 'bread', 'arugula', 'garlic']
[71, 1857, 56, 49, 57]
['salt', 'butter', 'asparagus', 'swiss cheese', 'eggs', 'shallots']
[71, 1857, 57, 1925, 452]
['bread crumbs', 'salt', 'provolone cheese', 'mustard', 'asparagus', 'mayonnaise', 'chicken breast', 'lemon', 'corn on the cob', 'pepper']
[71, 1857, 56, 452, 49]
['salt', 'olive oil', 'asparagus', 'pepper']
[53, 82, 654, 1768, 468]
['water', 'flour', 'broth', 'butter', 'asparagus', 'quesadillas', 'egg yolks', 'lemon juice', 'parsley', 'feta cheese', 'quinoa', 'oil', 'eggs', 'smoked salmon', 'garlic', 'onions', 'pepper']
[66, 1926, 46

[146, 805, 802, 1822, 804]
['corn chips', 'bacon', 'beer', 'zzseasoning', 'garlic', 'onions', 'bell peppers', 'sausage', 'sugar', 'pepper', 'basil', 'cheddar cheese', 'chili', 'barbecue sauce', 'salt', 'oregano', 'celery', 'beef', 'ground beef', 'tomatoes']
[1747, 4, 547, 294, 670]
['sugar', 'syrup', 'almonds', 'milk', 'seafood']
[1592, 1975, 1606, 1604, 1594]
['bread crumbs', 'butter', 'corn', 'cream cheese', 'salsa', 'tacos', 'cheese', 'ground beef', 'oil', 'pasta', 'black beans', 'tomatoes', 'barbecue sauce', 'garlic', 'onions', 'pepper']
[594, 214, 598, 151, 1436]
['pecans', 'flour', 'salt', 'butter', 'sugar', 'protein powder', 'syrup', 'zzseasoning', 'oil', 'milk', 'ice cream soda', 'corn on the cob']
[1470, 857, 950, 1972, 1788]
['salt', 'olive oil', 'pesto', 'pumpkin seeds', 'cherry pie', 'tofu', 'quinoa', 'shallots', 'kale', 'corn']
[1266, 409, 433, 384, 1389]
['salt', 'butter', 'orange', 'cream', 'syrup', 'brown sugar', 'zzseasoning', 'bread', 'eggs']
[262, 245, 1986, 1084, 20

[1211, 145, 1554, 624, 1669]
['rolls', 'provolone cheese', 'sausage', 'pepperoni', 'ground beef', 'onions', 'olives', 'mushroom', 'parmesan cheese', 'barbecue sauce', 'italian dressing', 'pepper']
[1340, 1750, 1433, 1634, 1777]
['salt', 'water', 'flour', 'sour cream', 'oregano', 'parsley', 'chicken breast', 'cheddar cheese', 'tomatoes', 'chili', 'garlic', 'onions', 'barbecue sauce', 'pepper']
[532, 1215, 1651, 303, 271]
['salt', 'olive oil', 'bell peppers', 'oregano', 'burgers', 'corn', 'celery', 'ground beef', 'zzseasoning', 'green beans', 'black beans', 'tomatoes', 'chili', 'garlic', 'onions', 'kidney beans', 'pepper']
[9, 663, 167, 1494, 660]
['salt', 'olive oil', 'wine', 'oregano', 'sausage', 'tomatoes', 'garlic', 'onions', 'pepper']
[281, 538, 956, 1651, 303]
['flour', 'colby cheese', 'refried beans', 'ground beef', 'olives', 'tomatoes', 'chili', 'garlic', 'onions', 'barbecue sauce']
[151, 214, 1634, 598, 1436]
['salt', 'mustard', 'bell peppers', 'macaroni', 'celery', 'sugar', 'vi

[83, 194, 585, 1863, 1969]
['cream cheese', 'vegetable oil', 'protein powder', 'mushroom', 'parmesan cheese', 'garlic', 'pepper']
[119, 1596, 2208, 2222, 150]
['mozzarella cheese', 'sour cream', 'cream cheese', 'bacon', 'ketchup', 'ground beef', 'cheddar cheese', 'mayonnaise', 'barbecue sauce', 'garlic', 'onions']
[1740, 1483, 1742, 1503, 1750]
['olive oil', 'egg whites', 'butter', 'corn', 'pepper', 'thyme', 'parsley', 'oil', 'mushroom', 'bread', 'garlic', 'onions', 'broth']
[585, 1969, 194, 1927, 1632]
['water', 'flour', 'mustard', 'vegetable oil', 'soy sauce', 'ground beef', 'potatoes', 'cheddar cheese', 'chili', 'garlic', 'onions', 'kidney beans', 'pepper']
[1964, 2184, 2194, 571, 1]
['rye bread', 'butter', 'swiss cheese', 'ground beef', 'barbecue sauce', 'onions', 'pepper']
[598, 151, 1436, 1634, 903]
['olive oil', 'butter', 'wine', 'parmesan cheese', 'shrimp', 'basil', 'parsley', 'lemon', 'pasta', 'barbecue sauce', 'garlic', 'onions', 'pepper']
[1353, 1554, 138, 539, 471]
['salt',

...

[1281, 1274, 1282, 1125, 795]
['strawberries', 'salt', 'cream', 'lemon juice', 'sugar', 'wine']
[1278, 2039, 947, 1288, 1294]
['fudge', 'cream', 'chocolate chip cookies', 'margarine', 'cherries']
[2026, 1478, 838, 847, 1526]
['strawberries', 'rum', 'cream', 'sugar', 'zzseasoning', 'champagne']
[2039, 2037, 637, 1909, 1292]
['olive oil', 'mustard', 'bok choy', 'chicken', 'honey', 'protein powder', 'soy sauce', 'duck', 'onions']
[2101, 407, 1447, 548, 33]
['olive oil', 'mustard', 'butter', 'wine', 'plums', 'thyme', 'sugar', 'beef', 'shallots', 'duck']
[1993, 941, 516, 2251, 14]
['salt', 'garlic', 'brussels sprouts', 'corn', 'salsa', 'basil', 'balsamic vinegar', 'cabbage', 'zzseasoning', 'cherries', 'goat cheese', 'chili', 'duck', 'onions', 'pepper']
[2069, 2071, 2056, 2067, 2062]
['garlic', 'corn', 'ketchup', 'chicken', 'protein powder', 'sugar', 'soy sauce', 'seeds', 'vinegar', 'ginger', 'rice noodles', 'eggs', 'spinach (cooked)', 'mixed vegetables']
[134, 2019, 1743, 1876, 1437]
['salt

[629, 2099, 627, 630, 1247]
['bread crumbs', 'salt', 'olive oil', 'mustard', 'lamb', 'rosemary', 'garlic', 'pepper']
[1558, 2103, 1579, 2109, 1773]
['salt', 'olive oil', 'cilantro', 'sugar', 'vinegar', 'ground beef', 'ice cream cones', 'tomatoes', 'corn on the cob', 'rice', 'onions', 'pepper']
[1349, 1753, 1777, 1486, 1750]
['lemon', 'wine', 'garlic', 'onions', 'water', 'butter', 'egg yolks', 'cheese', 'potatoes', 'ice cream cones', 'corn on the cob', 'pepper', 'lemon juice', 'milk', 'salt', 'olive oil', 'flour', 'eggplant', 'oregano', 'beef', 'ground beef', 'tomatoes']
[516, 14, 2251, 1024, 941]
['salt', 'flour', 'water', 'wine', 'bacon', 'thyme', 'sugar', 'lamb', 'carrot', 'beef', 'potatoes', 'zzseasoning', 'garlic', 'onions', 'pepper']
[2103, 1558, 2106, 1579, 1773]
['thyme', 'lamb', 'parsley', 'wine', 'onions']
[2101, 1447, 548, 407, 33]
['olive oil', 'butter', 'chicken', 'thyme', 'basil', 'balsamic vinegar', 'lamb chops', 'rosemary', 'shallots', 'pepper']
[654, 468, 1768, 53, 82]


[1933, 884, 1066, 1803, 1810]
['olive oil', 'mussels', 'wine', 'oregano', 'squid', 'bread', 'shrimp', 'grape juice', 'zzseasoning', 'fennel', 'haddock', 'tomatoes', 'garlic', 'onions', 'pepper']
[2162, 1669, 1554, 898, 138]
['salt', 'crab', 'butter']
[1593, 2165, 2173, 2170, 2172]
['olive oil', 'mussels', 'wine', 'oregano', 'squid', 'bread', 'shrimp', 'grape juice', 'zzseasoning', 'fennel', 'haddock', 'tomatoes', 'garlic', 'onions', 'pepper']
[2162, 1669, 1554, 898, 138]
['water', 'butter', 'vinegar', 'corn on the cob', 'blue cheese']
[1937, 1068, 1082, 854, 707]
['salt', 'flour', 'butter', 'chive', 'vegetable oil', 'crab', 'lemon juice', 'vinegar', 'eggs', 'bread', 'milk', 'arugula', 'barbecue sauce']
[1694, 98, 1464, 1715, 1465]
['salt', 'mustard', 'crab', 'egg yolks', 'sugar', 'oil', 'lemon', 'shallots', 'garlic', 'pepper']
[1593, 2161, 2173, 2170, 2172]
['olive oil', 'crab', 'parsley', 'zzseasoning', 'lemon', 'red potatoes', 'garlic']
[1407, 62, 1023, 1253, 730]
['salt', 'olive oil

[2205, 2207, 2224, 1699, 891]
['mozzarella cheese', 'olive oil', 'tortellini', 'pepperoni', 'balsamic vinegar', 'vinegar', 'italian dressing', 'olives', 'artichokes', 'onions', 'pepper']
[127, 454, 1342, 1869, 634]
['rolls', 'provolone cheese', 'bell peppers', 'swiss cheese', 'salami', 'pepper', 'pepperoni', 'parmesan cheese', 'eggs', 'ham']
[492, 1635, 273, 1354, 530]
['mustard', 'oregano', 'vegetable oil', 'thyme', 'pepperoni', 'frozen yogurt', 'scrambled eggs', 'cottage cheese', 'pepper']
[1791, 1979, 1763, 1201, 1748]
['pesto', 'olive oil', 'mozzarella cheese', 'salami', 'zzseasoning']
[809, 1888, 1891, 1894, 1895]
['salami', 'bell peppers', 'cream cheese']
[119, 1596, 2208, 210, 812]
['flour', 'mustard', 'wine', 'salami', 'cheese', 'potatoes', 'zzseasoning', 'garlic']
[2072, 1847, 1629, 425, 1841]
['flour', 'mozzarella cheese', 'oregano', 'salami', 'pizza', 'zzseasoning', 'bread', 'onions']
[1699, 2207, 2232, 1697, 1695]
['mozzarella cheese', 'olive oil', 'salami', 'spinach (raw)'

NameError: name 'sr' is not defined

In [257]:
(stop - start)/60

datetime.timedelta(seconds=55, microseconds=141022)

In [258]:
import json
ts = datetime.now().strftime("%Y_%m_%d_%H_%M")
json_file_nm = "RR_results_"+ts+".json"
res = recommendations.to_json(orient='records')
parsed = json.loads(res)

In [260]:
# convert to reciepe_id based
all_recipes = {}
for i in parsed:
    if i['current_recipe'] in all_recipes.keys():
        all_recipes.get(i['current_recipe']).append(i)
    else:
        all_recipes[i['current_recipe']] = [i]

In [261]:
with open(json_file_nm, "w") as outfile:
    json.dump(all_recipes, outfile, indent=4) 